# Search for popular skills needed for data scientist in vn.indeed.com

Step 1: Make a list of link to each job

Step 2: Go to each link and scrape for information

Step 3: Make stats comparision

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import re

In [2]:
# Study the web

url = 'https://vn.indeed.com/Vi%E1%BB%87c-l%C3%A0m?q=data+science&l='
source_doc = requests.get(url).text
source_tree = BeautifulSoup(source_doc, 'html.parser')

In [4]:
#How many jobs posted

job_count_string = source_tree.find(id = 'searchCount').contents[0]
job_count_string = job_count_string.split()[-1]
print('Search yielded {} hit'.format(job_count_string))
job_count = int(job_count_string)

Search yielded 370 hit


In [5]:
'''Go to each page and scrape'''

num_pages = int(job_count // 10)

base_url = 'https://vn.indeed.com'
job_links = []

for i in range(num_pages + 1):
    if i == 0:
        url = 'https://vn.indeed.com/Vi%E1%BB%87c-l%C3%A0m?q=data+science&l='
    else:
        url = 'https://vn.indeed.com/Vi%E1%BB%87c-l%C3%A0m?q=data+science&start=' + str(i*10)
    
    html_page = requests.get(url).text
    bs_tree = BeautifulSoup(html_page, 'html.parser')
    job_area = bs_tree.find(id = 'resultsCol')
    job_post = job_area.findAll('div')
    job_post = [jp for jp in job_post if not jp.get('class') is None
               and ''.join(jp.get('class')) == 'rowresult']
    job_ids = [jp.get('data-jk') for jp in job_post]
    
    for ids in job_ids:
        job_links.append(base_url + '/viewjob?jk=' + ids)
        
    time.sleep(1)

print('We found {} jobs'.format(len(job_links)))

We found 397 jobs


In [10]:
'''
Go to each link and scrape for information
'''

languages = {'python': 0, "r": 0, 'java': 0, 'php':0}
db_skills = {'sql': 0, 'mysql':0, 'mongodb': 0}
degrees = {'bachelor': 0, 'ph.d': 0, 'master': 0}

def requirements(requirement):
    counter = 1
    for link in job_links:
        try:
            html_page = requests.get(link).text
        except:
            return

        html_text = BeautifulSoup(html_page, 'html.parser')
        
        for script in html_text(["script", "style"]):
            script.extract()
        text = html_text.get_text()
        
        for key in requirement.keys():
            if key in text.lower():
                requirement[key] += 1
        counter += 1
        time.sleep(1)
    
    if counter % 10 == 0:
        print('Scanned {} job'.format(counter))
    
    return requirement

result = requirements(degrees)

print(result)

{'bachelor': 166, 'ph.d': 5, 'master': 60}


In [ ]:
'''Make statistics comparision
'''